In [1]:
from lxml import html
import requests
import re
import json
import numpy as np
#from xml.etree import ElementTree as etree

In [2]:
page = requests.get('http://www.textart.ru/database/slogan/list-advertising-slogans.html')
tree = html.fromstring(page.content)

In [3]:
subcategories_links = tree.xpath("//div/select/option/@value")

In [4]:
regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

In [5]:
strs = ["aaaeeeooouuuu  o assss", "eounmvc","hoooieunmcv","ttlghj"]

In [21]:
small_letters = "weruoaszxcvnm"
big_letters = "qtyipdfghjklb"

In [6]:
for s in strs:
    print(re.match("[weruoaszxcvnm ]+", s).group())
    print(re.match("[qtyipdfghjklb]+", s).group())

aaaeeeooouuuu  o assss


AttributeError: 'NoneType' object has no attribute 'group'

In [7]:
def get_slogans_from_page(tree):
    slogan_dict = {}
    for paragraph in tree.xpath("//p[contains(@class, 'paragraf')]"):
        brand = paragraph.text
        slogan = paragraph.find("span")
        if brand is None or slogan is None:
            continue
        
        final_slogan = slogan.text.strip()
        final_slogan = re.sub("\n+", "", final_slogan)
        final_slogan = re.sub("^\s+", "", final_slogan)
        final_slogan = re.sub("\s+", " ", final_slogan)
        slogan_dict[brand] = final_slogan
    return slogan_dict
get_slogans_from_page(tree)

{}

In [8]:
def get_category_name(tree_element):
    cat = tree_element[-1] \
                        .strip().removesuffix(' slogans.') \
                        .removesuffix(' slogans') \
                        .removesuffix(' advertising') \
                        .removeprefix("Database of slogans. ") \
                        .removeprefix("Advertising slogans for ") \
                        .removesuffix(".") \
                        .capitalize()
    cat = re.sub("Database\s+ of slogans\.\s+.*?", "", cat)
    cat = re.sub("\n", "", cat)
    cat = re.sub("\s+", " ", cat)
    return cat.capitalize()

In [9]:
def verify_link(link):
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, link) is not None

In [10]:
slogans = {}
for link in subcategories_links[1:4]:
    
    if re.match(regex, link) is None:
        continue
        
    page = requests.get(link)
    tree = html.fromstring(page.content)
    
    if not tree.xpath("//h1/font/text()"):
        continue
    
    #print(tree.xpath("//h1/font/text()")[0])
#    category = re.search("Database\s+?\n+\s+of slogans. (.*)\s+", tree.xpath("//h1/font/text()")[0].strip() \
#                         .removesuffix("advertising slogans."), re.MULTILINE).groups()[0]
    category = get_category_name(tree.xpath("//h1/font/text()"))
    
    slogans[category] = {}
    
    for sublink in tree.xpath("//div/select/option/@value"):
        if re.match(regex, sublink) is None:
            continue
        
        page = requests.get(sublink)
        tree = html.fromstring(page.content)
        subcategory = get_category_name(tree.xpath("//h1/text()"))
        
        
        if subcategory == "":
            print(sublink)
            subcategory = get_category_name(tree.xpath("//h1/font/text()"))

            slogans[category][subcategory] = {}
            for sub_sublink in tree.xpath("//div/select/option/@value"):
                if re.match(regex, sub_sublink) is None:
                    continue
                
                page = requests.get(sub_sublink)
                tree = html.fromstring(page.content)
                sub_subcategory = get_category_name(tree.xpath("//h1/font/text()"))
                print(sub_subcategory)
                if not sub_subcategory:
                    print("Bazinga")
                    continue
                
                
                slogans[category][subcategory][sub_subcategory] = {}
                slogans_dict = get_slogans_from_page(tree)
                slogans[category][subcategory][sub_subcategory] = slogans_dict
        else:

            #print(category, subcategory)
            
            slogans[category][subcategory] = {}
            slogans_dict = get_slogans_from_page(tree)
        
        #more than 1 page
        for additional_page in tree.xpath("//font/a/@href"):
            page = requests.get(additional_page)
            tree = html.fromstring(page.content)
            slogans_dict += get_slogans_from_page(tree)
            
        slogans[category][subcategory] = slogans_dict

TypeError: unsupported operand type(s) for +=: 'dict' and 'dict'

In [ ]:
page = requests.get('http://www.textart.ru/database/slogan/list-advertising-slogans.html')
tree = html.fromstring(page.content)
get_all_links(tree)

In [ ]:
def get_all_links(tree):
    slogans = {}
    for link in tree.xpath("//div/select/option/@value"):
        #print(verify_link(link), link)
        if not verify_link(link):
            continue
        
        page = requests.get(link)
        tree = html.fromstring(page.content)
        category = get_category_name(tree.xpath("//h1/text()"))
            
        if category == "":
            #print(verify_link(link), link)
            continue
        
        slogans[category] = get_slogans_from_page(tree)
        #slogans_dict = get_slogans_from_page(tree)
        
        #more than 1 page
        for additional_page in tree.xpath("//font/a/@href"):
            page = requests.get(additional_page)
            tree = html.fromstring(page.content)
            slogans = {**slogans, **get_slogans_from_page(tree)}
    return slogans

In [15]:
def get_random_slogan():
    return np.random.choice(list(slogans['Apparel']['Beachwear & swimwear'].values()))

In [16]:
np.random.choice(list(slogans['Apparel']['Beachwear & swimwear'].values()))

'Now go get wet!'

In [17]:
list(slogans['Apparel']['Beachwear & swimwear'].values())

['Fueled by water.',
 'Beachwear competence.',
 'Arena. Water instinct.',
 'Beauty in all its forms.',
 'Always in front.',
 'Love the beach.',
 'All girls are gorgeous in Jantzen.',
 'Look 10 lbs lighter in 10 seconds.',
 'Live in water.',
 'Making women feel sexy since 1984.',
 'Now go get wet!',
 'Share our passion for swimming!',
 'If you doubt yourself, wear something else!',
 'Swimwear that sticks!',
 'All you need to reach the beach!',
 'Dive into colour.',
 'Virtual skin.',
 'Look good. Swim fast.',
 'Be beautiful. Be you.',
 'Worldwide beach culture.',
 'Custom fit. Custom fabulous.',
 'The leader in sun protection.',
 'Sun. Safe. Style.',
 'Look better nearly naked.',
 'Always look your best, even undressed!']

In [18]:
get_random_slogan()

'Now go get wet!'

In [19]:
with open("slogan_dict.json", "w") as fp:
    json.dump(slogans, fp)

In [20]:
slogans

{'Apparel': {'Beachwear & swimwear': {'Speedo Swimwear': 'Fueled by water.',
   'TA-BOU Beachwear': 'Beachwear competence.',
   'Arena, sport swimwear, gear for swimming': 'Arena. Water instinct.',
   'Arena Bodylift, body shaping swimsuits': 'Beauty in all its forms.',
   'TYR Swimwear': 'Always in front.',
   'Malibu Beachwear & Bikinis in Malaysia': 'Love the beach.',
   'Jantzen, swimming suits and beach wear': 'All girls are gorgeous in Jantzen.',
   'Miraclesuit Swimsuits': 'Look 10 lbs lighter in 10 seconds.',
   'Dolphin Swimwear': 'Live in water.',
   'Ujena Swimwear and Bikinis': 'Making women feel sexy since 1984.',
   'Splashgear Swimwear': 'Now go get wet!',
   'Zoggs, brand of goggles and swimwear': 'Share our passion for swimming!',
   "AussieBum, men's swimwear, surfwear and underwear": 'If you doubt yourself, wear something else!',
   'Hive Swimwear Shop': 'Swimwear that sticks!',
   'Wings Beachwear Stores in Florida': 'All you need to reach the beach!',
   'Maru Swim